# Install SReT Transformer

In [ ]:
!pip install git+https://github.com/rwightman/pytorch-image-models.git
!pip install einops

!git clone https://github.com/szq0214/SReT.git temp
!cp -r temp/* .
!rm -rf temp

# Import Libs

In [1]:
# Data EDA
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Model Training
import copy
import time
import torch
from skimage import io
from skimage.color import gray2rgb
from torchvision import transforms, utils
from tqdm import tqdm

# SReT Transformer
import SReT

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
ARTISTS_CSV_PATH = Path('../input/best-artworks-of-all-time/artists.csv')
IMAGES_PATH = Path('../input/best-artworks-of-all-time/images/images')

# Data EDA

In [3]:
df_artists = pd.read_csv(str(ARTISTS_CSV_PATH))
df_artists.head()

**Check if folders exists.**

In [4]:
for name in df_artists['name']:
    image_author_path = IMAGES_PATH / name.replace(' ', '_').replace('ü', 'u╠ê')
    if image_author_path.exists():
        print(f'Exists! {image_author_path}')
    else:
        print(f'WTF! {image_author_path}')

**Check paintings quantity by each author.**

In [6]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
plt.xticks(rotation=90)
sns.barplot(data=df_artists, x='name', y='paintings')

**Check quantity of authors with painting more or equal than 100.**

In [8]:
np.sum(df_artists['paintings'] >= 100)

# Prepare Data for model training

**Only keep authors with more than 100 paintings.**

In [ ]:
df_artists_model = df_artists.drop(df_artists[df_artists['paintings'] < 100].index)
print(f'Records: {len(df_artists_model)}')
df_artists_model.head()

**Create Pytorch dataset model.**

In [ ]:
class ArtistsPaintingsDataset(torch.utils.data.Dataset):
    def __init__(self, df_artists: pd.DataFrame, images_path: Path, min_author_idx:int, max_author_idx:int, transform=None):
        self.df_artists = df_artists
        self.images_path = images_path
        self.transform = transform
        self.min_author_idx = min_author_idx
        self.max_author_idx = max_author_idx

    def __len__(self):
        return (self.max_author_idx - self.min_author_idx) * len(self.df_artists)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        image_quantity_per_author = self.max_author_idx - self.min_author_idx
        author_id = idx // image_quantity_per_author
        author_image_id = idx % image_quantity_per_author + self.min_author_idx + 1 
        
        author_name = self.df_artists['name'].iloc[author_id].replace(' ', '_').replace('ü', 'u╠ê')
        image_path = self.images_path / author_name / f'{author_name}_{author_image_id}.jpg'
        image = io.imread(image_path)

        if len(image.shape) == 2 or image.shape[2] == 1:
            image = gray2rgb(image)
        
        if self.transform:
            image = self.transform(image)
        
        sample={'image': image, 'author_id': torch.from_numpy(np.array(int(author_id)))}
        return sample

**Intialize Train/Test datasets and dataloaders with simple augmentations.**

In [ ]:
transforms_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomHorizontalFlip(0.5),
    transforms.Resize(380),
    transforms.RandomCrop(380)
#     transforms.Resize(224),
#     transforms.RandomCrop(224)
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.Resize(380),
    transforms.RandomCrop(380)
#     transforms.Resize(224),
#     transforms.RandomCrop(224)
])



batch_size = 128

train_dataset = ArtistsPaintingsDataset(df_artists=df_artists_model, images_path=IMAGES_PATH, min_author_idx=0, max_author_idx=75, transform=transforms_train)
test_dataset = ArtistsPaintingsDataset(df_artists=df_artists_model, images_path=IMAGES_PATH, min_author_idx=75, max_author_idx=100, transform=transforms_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

dataset_sizes = {'train': len(train_dataset), 'test': len(test_dataset)}

# Model Training

**Implement training function.**

In [ ]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for samples in tqdm(dataloaders[phase]):
                inputs = samples['image'].to(device)
                labels = samples['author_id'].to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best test Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

**Download model for training (need to uncomment a model which you want to train).**

In [ ]:
# # EFFICIENTNET-B4
# model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b4', pretrained=True)

# # RESNET-50
# model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)

# # SReT_S
# model = SReT.SReT_S(pretrained=False)
# model.load_state_dict(torch.load('../input/sret-s-pth/SReT_S.pth')['model'])

**Freeze/unfreeze some layers for training (need to uncomment a model which you want to train).**

In [ ]:
for param in model.parameters():
    param.requires_grad = False

# # EFFICIENTNET-B4
# model.features.requires_grad_(True)
# model.classifier.requires_grad_(True)
# model.classifier.fc = torch.nn.Linear(1792, 30)

# # RESNET-50
# model.fc = torch.nn.Linear(2048, 30)

# # SReT_S
# for idx, param in enumerate(model.transformers[2].parameters()):
#     if idx > 80: 
#         param.requires_grad = True
# model.norm.requires_grad_(True)
# model.avgpool.requires_grad_(True)
# model.head.requires_grad_(True)
# model.head = torch.nn.Linear(504, 30)

model.cuda()
print()

**Running a training.**

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.33)
best_model = train_model(model, criterion, optimizer, scheduler, {'train': train_loader, 'test': test_loader}, num_epochs=15) 

**Saving model weights.**

In [ ]:
# # EFFICIENTNET-B4
# torch.save(model.state_dict(), 'efficient_b4_03.pt')

# # RESNET-50
# torch.save(model.state_dict(), 'resnet_50_01.pt')

# # SReT_S
# torch.save(model.state_dict(), 'sret-s_01.pt')

**Saving authors dataframe which were taken for the training.**

In [ ]:
df_artists_model.to_csv('artists.csv', index=False)